In [ ]:
import os
os.chdir('../..')
from pipelines.util import *
import pandas as pd
from datetime import datetime
from collections import Counter
import re

In [ ]:
data = pd.read_csv(os.path.join(WDIR, 'true-north/true_north_clean.csv'))
data.drop(columns='associated Company Domain (Contact Level)', inplace=True)
# data[data['Industry'].isna()]

In [ ]:
# create empty dataframe with the same columns as `data`
summary = pd.DataFrame(columns=data.columns.to_list())
summary.loc['count'] = data.count()
mode = data.mode().head(1)
mode.reset_index(drop=True, inplace=True)
mode.rename(index={0: 'top'}, inplace=True)
summary = pd.concat([summary, mode])
freq = data.apply(lambda x: x.value_counts(dropna=True).max())
summary.loc['freq'] = freq

unique = data.nunique(dropna=True)
summary.loc['unique'] = unique

summary

In [ ]:
# summary = data.describe()
summary.loc['top_percent_of_count'] = (summary.loc['freq'] * 100 / summary.loc['count']).astype(float).round(1)

In [ ]:
summary

In [ ]:
company_names = pd.Series(data['Company Name'].sort_values())
print(company_names.unique())

There is LLoyds banking group and lloyds bank plc - we're assuming these are different companies.

In [ ]:
# convert the month column to a datetime object
data['month'] = pd.to_datetime(data['Create Date'])

#convert the item to a formatted value in yyyy-mm format.
data['month_formatted'] = data['month'].apply(datetime.strftime, format='%Y-%m').sort_index(ascending=True)

In [ ]:
# take the number of members and calculate the number that joined each month.
monthly_members = pd.DataFrame(data['month_formatted'].value_counts(ascending=False)).reset_index()
monthly_members['start_of_month'] = monthly_members['month_formatted'].astype(str) + '-01'

In [ ]:
monthly_orgs = data.drop_duplicates(subset='Company Name', keep='last')
monthly_orgs = pd.DataFrame(monthly_orgs['month_formatted'].value_counts(ascending=False)).reset_index()
monthly_orgs['start_of_month'] = monthly_orgs['month_formatted'].astype(str) + '-01'

In [ ]:
def decimal_date(data):
    # make a unix timestamp column
    data['timestamp'] = pd.to_datetime(data['month_formatted'], format='%Y-%m').astype(int) / 10**9
    # make a decimal date and round to 2dp.
    data['year'] = data['timestamp'].div((86400*365.25)).add(1970).round(2)
    # drop the timestamp column
    data.drop(columns='timestamp', inplace=True) 
    # set year and formatted month as the index so they aren't included in the cumsum.
    data.set_index(['year', 'month_formatted', 'start_of_month'], inplace=True, append=True)

    return data

In [ ]:
def calculate_cumsum(data, count_name):
    # order by date, then do the cumsum. reset the index, drop the original index column as not needed
    data = pd.DataFrame(data.sort_index(level=2).cumsum(skipna=True).reset_index().drop(columns='level_0'))
    data.rename(columns={'count': f'{count_name}'}, inplace=True)
    return data

In [ ]:
# apply above functions to data
cs_monthly_members = calculate_cumsum(decimal_date(monthly_members), count_name='individuals')
cs_monthly_orgs = calculate_cumsum(decimal_date(monthly_orgs), count_name='orgs')

In [ ]:
# merged the two dataframes
cs_merged = cs_monthly_members.merge(cs_monthly_orgs, how='inner', on=['year', 'month_formatted', 'start_of_month'])

In [ ]:
# write to file
cs_merged.to_csv(os.path.join(SRC_DIR,'overview/membership/_data/cumsum.csv'), index=False)

Make a bar chart for locations.

In [ ]:
locations = data.copy()
split_locs = locations['location'].str.split(';').apply(pd.Series, 1).stack()
split_locs.index = split_locs.index.droplevel(-1)
split_locs.name = 'location'
del locations['location']
locations = locations.join(split_locs)
locations['location'] = locations['location'].str.lstrip()
top_locations = locations['location'].value_counts().reset_index().head(5)
top_locations.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/top_locations.csv'), index=False)

Calculating some summary stats

In [ ]:
total_members = len(data.index)

total_companies = summary.loc['unique', 'Company Name']

top_company_size = summary.loc['top', 'company_size']

top_company_size_pct = summary.loc['top_percent_of_count', 'company_size']

# top_industry = summary.loc['top', 'Industry']
top_industry = ''
top_industry_pct = 0
# top_industry_pct = summary.loc['top_percent_of_count', 'Industry']

In [ ]:
data['City']

### Word frequency in the sector column

In [ ]:
def normalize_string(s):
    # Convert to lowercase and remove non-alphanumeric characters (keeping spaces)
    try:
        s = s.lower()
    except:
        print(f"'{s}' is not a stirng type. Converting to string\n")
        return str(s)
    s = re.sub(r'[^a-z0-9\s]', '', s)
    return s
data['normalized_sector'] = data['sector'].apply(normalize_string).str.split(';')
 
# Flatten the list and further split by spaces to handle multi-word strings
all_words = [word for sublist in data['normalized_sector'] for item in sublist for word in item.split()]
 
# Count the occurrences of each word
word_counts = Counter(all_words)
 
# Find the most common word
most_common_words = word_counts.most_common(10)
least_common_words = word_counts.most_common()

banned_words = ['and', 'or', 'of', 'it', 'the', 'for', 'with', 'we', 'nan', 'please', 'specify']
# print("The 10 most common words, not including 'and' are:") 
# for word, count in most_common_words:
#     if word == banned_words:
#         continue
#     print(f"'{word}' with {count} occurences")

print("The most common words, with at least 2 occurences are:")
words = []
counts = []
for word, count in least_common_words:
    if word in banned_words:
        continue
    if word == 'estate':
        continue
    if count >= 10:
        if word == 'real':
            words.append('real estate')
        else:
            words.append(word)
        counts.append(count)
        print(f"'{word}' with {count} occurences")

sector_counts = pd.DataFrame(data={'name': words, 'count': counts}).set_index('name')
# sector_counts['colour'] = round((sector_counts['count'] - min(sector_counts['count'])) / sector_counts['count'].max(), 3)
sector_counts.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/sector_word_counts.csv'))
# sector_strings = [item for sublist in data['sector'].str.split(';') for item in sublist]
# string_counts = Counter(sector_strings)
# print(string_counts.most_common())


In [ ]:
advisory_council = 11

if summary.loc['top', "Are you currently a B Corp or in the process of becoming a B Corp?"] == 'No':
    b_corps_pct = round(100 - summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"], 1)
else:
    summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"]

membership_increase = cs_merged['individuals'].pct_change().mul(100).iloc[-1].round(1)

northern_stars = len(pd.read_csv(os.path.join(SRC_DIR, 'overview/northern-stars/_data/northern_stars.csv')))

names = ["Total members", 
         "Total companies", 
         "Membership increase", 
         f"Companies with {top_company_size} employees", 
         "Top industry", 
         "Geographic reach", 
         "Advisory council", 
         "Northern stars", 
         "B Corps"]

values = [total_members, 
          total_companies, 
          membership_increase, 
          top_company_size_pct, 
          top_industry_pct, 
          "4", 
          advisory_council, 
          northern_stars, 
          b_corps_pct]

footnotes = ["People", 
             "Unique companies", 
             "Since last month", 
             "Of members work in companies of this size", 
             f"Of members work in {top_industry}", 
             "placeholder", 
             "Members represent the network on the True North advisory council", 
             "Companies have been featured as Northern Stars", 
             "Of member's organisations are B Corps or are joining"]

posts = ['','','%','%','%','','','','%']

urls = ['/overview/membership', '/overview/membership', '/overview/membership', '', '', '', 'https://www.brabners.com/insights/true-north/true-north-advisory-council-launches', '/overview/northern-stars', '/themes/sustainable-growth/b-corporations']
dashboard = pd.DataFrame(data={'name':names, 'value': values, 'footnote': footnotes, 'post': posts, 'url': urls})

dashboard

In [ ]:
dashboard.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/true_north_members_list.csv'), index=False)

In [ ]:
time_updated('src/overview/membership/_data/updated.yaml')